## Backtesting
### Rendimiento de Carteras con Entradas y Salidas de Efectivo

Este cuaderno muestra cómo realizar la evaluación de carteras de inversión abiertas,
en las que pueden ocurrir entradas y salidas de capital invertido.

Aunque muchos de los algoritmos de trading se evaluan sobre un capital inicial,
en la realidad las entradas y salidas de capital invertido es una situación 
muy habitual, por ejemplo como sucede con los fondos de inversión abiertos

___

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

____
### Datos

In [ ]:
ticker_list = ['BBVA','SAN','REP','TEF','IBE','FER','ITX','ACS','AMS','GRF']

In [ ]:
import pickle
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

In [ ]:
close_series = {ticker: df.close
                for ticker, df in stock_data.items()
                if ticker in ticker_list
               }
stock_df = pd.DataFrame(close_series)
stock_df = stock_df.loc['2016':]

In [ ]:
stock_df.head()

___

### Rentabilidad Ponderada por Tiempo
- En inglés *Time-Weighted Return* (TWR) 
- Es el estándar de la indusria para medir rendimiento cuando hay flujos de caja externos al capital invertido inicialmente
- Equivale al rendimiento que tendría 1€ a lo largo de todo el periodo evaluado, independientemente del dinero que entra y sale

#### Procedimiento de Calculo
El principio básico es componer la rentabilidad de cada unidad de tiempo (ej. días). Sin embargo es suficiente
subdividir en periodos en los que no haya flujos de caja externo. El procedimiento consiste en los siguientes pasos:
1. Se subdivide el período analizado en subperiodos, haciendo un corte siempre que haya un flujo de caja
2. Se calcula la rentabilidad de cada subperiodo considerando la valoración sobre el capital invertido en dicho subperiodo
3. Componer la rentabilidad de cada subperiodo

$$ \rm{TWR} = \prod_{i=1}^n (1 + HPR_i) - 1 $$

donde
- TWR: Time-weighted return
- HPR_i: *Holding-period return*, la rentabilidad de cada sub-periodo $\frac{P_f}{P_i} - 1$ 

____

#### Ejemplo TWR

- Iniciamos una cartera con 100mil euros a principios de año
- El 1 de junio la cartera vale 105mil euros y recibimos una inversión adicional de 20mil
- El 1 de agosto la cartera vale 95 y se sacan 35mil euros
- El 31 de diciembre la cartera vale 90 mil.

Calculamos en miles. Tenemos 3 periodos:
1. $\rm{HPR_1} = \frac{105 - 100}{100} = 0.05$  
Para el siguiente periodo la inversión es $105+20=125$ 

2. $\rm{HPR_2} = \frac{95 - 125}{125} = -0,24$  
Para el siguiente periodo la inversión es $95-35=60$

3. $\rm{HPR_3} = \frac{90 - 60}{60} = 0.50$  


La rentabilidad ponderada por tiempo es:

$ \rm{TWR} = (1.05)(0.76)(1.5) - 1 = 0.197 $

____

#### TWR con valoracion de participaciones
Una forma equivalente de calcular el TWR es asumir que las inversiones corresponden a un **número participaciones** 
que deben tener los mismos beneficios a lo largo del tiempo. (Recordad que el TWR equivale a la rentabilidad de 1€ invertido con independencia de entradas y salidas).  Esto implica que entradas y salidas de capital corresponden con compras y ventas de participaciones al valor liquidativo del día del flujo de entrada/salida.

Este es el mecanismo que utilizan los fondos de inversión abiertos.
- Las entradas corresponden a suscripciones de nuevas participaciones
- Las salidas corresponden a reembolsos de participaciones

El procedimiento consiste en:
1. Calcular el valor liquidativo de una participación. Esto es patrimonio entre número de participaciones
2. Suscripciones y reembolsos se compran o venden a valor liquidativo
3. La rentabilidad se calcula sobre el valor liquidativo

___

#### Ejemplo con valoración de participaciones

Usamos el mismo escenario del ejemplo anterior y partimos de unas 1000 participaciones
valoradas inicialmente a 100€. 

1. El 1 de junio la cartera vale 105mil euros y recibimos una inversión adicional de 20mil
  - valor liquidativo $\rm{VL} = \frac{105000}{1000} = 105$
  - suscripción de $\frac{20000}{105} = 190.47619$ nuevas participaciones
  - total de participaciones 1190.47619

2. El 1 de agosto la cartera vale 95 y se sacan 35mil eurosm
  - valor liquidativo $\rm{VL} = \frac{95000}{1190.47619} = 79.8$
  - reembolso de $\frac{35000}{79.8} = 438.59649$ participaciones
  - total de participaciones $1190.47619 - 438.59649 = 751.8797$

3. El 31 de diciembre la cartera vale 90 mil.
  - valor liquidativo $\rm{VL}= \frac{90000}{751.8797} = 119.7$
  - Rentabilidad de la cartera $ \rm{TWR} = \frac{119.7}{100} - 1 = 0.197$

El resultado, tal como esperábamos es el mismo.

___

In [ ]:
ticker = 'ACS'
stock_series = stock_df[ticker]
stock_series.plot()

In [ ]:
# inversion inicial
capital = 100_000

In [ ]:
# valor liquidativo, inicial=100
nav = pd.Series(100, index=stock_series.index)
nav

In [ ]:
# participaciones
init_n_units = capital/nav.iloc[0]
init_n_units

In [ ]:
# registro de suscripciones y reembolsos
inout_units = pd.Series(0, index=stock_series.index)
inout_units.iloc[0] = init_n_units

n_units = inout_shares.cumsum()
n_units

In [ ]:
# delta_cash = pd.Series(0, index=stock_df.index)
delta_shares = pd.DataFrame(0, index=stock_df.index, columns=stock_df.columns)
delta_shares[ticker].iloc[0] = capital/stock_series.iloc[0]
delta_shares.head()

In [ ]:
posiciones = delta_shares.cumsum()
posiciones

In [ ]:
asset_value = posiciones * stock_df
asset_value

In [ ]:
portvalue = asset_value.sum(axis=1)
portvalue

In [ ]:
nav = portvalue / n_units
nav.head()

dado que no tenemos ahora entradas y salidas de capital,
la evolución del nav, coincide con la evolución del portfolio

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,4))
portvalue.plot(ax=ax1)
nav.plot(ax=ax2)

In [ ]:
# serie booleana a partir de convertir a fecha mensual 
# y luego ver las repeditas
month_repeated = stock_series.index.to_period('m').duplicated()
month_repeated 

In [ ]:
first_month_trade = stock_series[~month_repeated]
first_month_trade.head()

In [ ]:
cash_flows = pd.Series(0, index=stock_df.index)
cash_flows.iloc[0] = -capital

In [ ]:
regular_cash_in = 2000

In [ ]:
for idate in first_month_trade.index[1:]:
    # suscripciones. las nuevas participaciones las tendremos de aquí en adelante
    new_units = regular_cash_in/nav.loc[idate]
    inout_units.loc[idate] = new_units
    n_units = inout_units.cumsum()
    
    # inversion del nuevo efectivo
    delta_shares.loc[idate, ticker] = regular_cash_in/stock_series[idate]
    
    # recalculamos la cartera
    posiciones = delta_shares.cumsum()
    asset_value = posiciones * stock_df
    portvalue = asset_value.sum(axis=1)
    nav = portvalue / n_units
    
    # registramos el cashflow
    cash_flows[idate] = -regular_cash_in    

In [ ]:
n_units.plot()

In [ ]:
portvalue

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,4))
portvalue.plot(ax=ax1)
nav.plot(ax=ax2)